<a href="https://colab.research.google.com/github/gabrielborja/python_data_analysis/blob/main/chocolate_making_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Particle Size Analytics

## Uploading necessary packages

In [ ]:
#Upgrading Plotly
!pip install plotly --upgrade

In [ ]:
#Uploading packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from ipywidgets import interact

In [ ]:
#Customizing data visualization styles
sns.set_theme(rc = {'figure.figsize': (8, 5)})

In [ ]:
#Remove previous versions of the uploaded excel file
!rm 2017_2021_mass_quality_data.xlsx
!rm 2019_2021_malvern_D_90.xlsx

In [ ]:
#Uploading file from local drive ==> !rm file
from google.colab import files
uploaded8_a = files.upload()

In [ ]:
#Storing dataset in a Pandas Dataframe
import io
df8_a = pd.read_excel(io.BytesIO(uploaded8_a['2017_2021_mass_quality_data.xlsx']))

In [ ]:
#Checking the first dataframe info
df8_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439 entries, 0 to 438
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Plant                 439 non-null    object        
 1   Material              439 non-null    int64         
 2   Material_Description  439 non-null    object        
 3   Physical_sample       439 non-null    int64         
 4   Production_date       439 non-null    datetime64[ns]
 5   Short_text            26 non-null     object        
 6   MDLZ_spec             439 non-null    float64       
 7   Heerenthals_spec      439 non-null    float64       
 8   Fat_content_%         439 non-null    float64       
 9   USL_particle_size_um  439 non-null    int64         
 10  Recorded_results      439 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(3)
memory usage: 37.9+ KB


In [ ]:
#Uploading file from local drive ==> !rm file
from google.colab import files
uploaded8_b = files.upload()

In [ ]:
#Storing dataset in a Pandas Dataframe
import io
df8_b = pd.read_excel(io.BytesIO(uploaded8_b['2019_2021_conch_measurements_full.xlsx']))

In [ ]:
#Checking the second dataframe info
df8_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3599 entries, 0 to 3598
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Material              3599 non-null   int64         
 1   Material_description  3599 non-null   object        
 2   PhysSample            3599 non-null   int64         
 3   Insp_Lot              3599 non-null   int64         
 4   Short_text            3599 non-null   object        
 5   Recorded_results      3509 non-null   float64       
 6   Sample_date           3599 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 196.9+ KB


##Data manipulation

In [ ]:
#Creating categories from datetime for first dataframe
df8_a = df8_a.assign(Year = df8_a['Production_date'].dt.year,
                 Month = df8_a['Production_date'].dt.month_name(),
                 Quarter = df8_a['Production_date'].dt.quarter,
                 Weeknum = df8_a['Production_date'].dt.isocalendar().week)

In [ ]:
#Creating categories from datetime for first dataframe
df8_b = df8_b.assign(Year = df8_b['Sample_date'].dt.year,
                     Month = df8_b['Sample_date'].dt.month_name(),
                     Quarter = df8_b['Sample_date'].dt.quarter,
                     Weeknum = df8_b['Sample_date'].dt.isocalendar().week)

In [ ]:
#Transforming quarters into categories
quarter_dict = {1: 'Q1', 2: 'Q2', 3: 'Q3', 4: 'Q4'}
df8_a['Quarter'].replace(quarter_dict, inplace=True)
df8_b['Quarter'].replace(quarter_dict, inplace=True)

In [ ]:
#Creating list of categories
year_list = list(sorted(df8_a['Year'].unique(), reverse=True))
#month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'Jul', 'August', 'September', 'October', 'Novemeber', 'December']
quarter_list = list(df8_a['Quarter'].unique())
period_list = ['Month', 'Quarter', 'Weeknum']
cat_list = ['Recorded_results', 'Fat_content_%']

In [ ]:
#Aggregating dataframe 1 with avergage values
df8_a_g = df8_a.groupby(by=['Year', 'Weeknum']).agg(Fat_avg = ('Fat_content_%', 'mean'), Malvern_avg = ('Recorded_results', 'mean')).reset_index()

In [ ]:
#Aggregating dataframe 2 with avergage values
df8_b_g = df8_b[df8_b['Short_text']=='PS; Malvern D 90'].groupby(by=['Year', 'Weeknum']).agg(Conch_avg = ('Recorded_results', 'mean')).reset_index()

In [ ]:
#Merging dataframe 1 and 2 on Year and Weeknum
df8_merged = df8_a_g.merge(df8_b_g, on=['Year', 'Weeknum'])

In [ ]:
#Checking the merged dataframe tail
df8_merged.tail(12)

,Year,Weeknum,Fat_avg,Malvern_avg,Conch_avg
109,2021,34,54.680000,29.185000,29.860000
110,2021,35,54.175000,26.242500,30.500000
111,2021,36,53.990000,28.850000,30.175000
112,2021,37,54.316667,25.330000,29.516667
113,2021,38,53.922500,26.217500,29.500000
114,2021,39,54.470000,27.050000,29.857143
115,2021,40,54.377500,25.475000,28.825000
116,2021,41,54.133333,24.926667,29.100000
117,2021,42,53.753333,27.620000,30.233333
118,2021,43,54.285000,25.675000,30.366667


In [ ]:
#Correlation matrix of merged variables
df8_merged.corr()

,Year,Weeknum,Fat_avg,Malvern_avg,Conch_avg
Year,1.000000,-0.108695,0.258201,-0.657037,0.117723
Weeknum,-0.108695,1.000000,-0.338801,0.200783,-0.032219
Fat_avg,0.258201,-0.338801,1.000000,-0.231841,0.037430
Malvern_avg,-0.657037,0.200783,-0.231841,1.000000,0.190715
Conch_avg,0.117723,-0.032219,0.037430,0.190715,1.000000


##Particle Size Interactive Plot

In [ ]:
#Creating interactive visualization
@interact(Year=year_list, Period=period_list, Cat=cat_list)
def plot_particle_size(Year, Period, Cat):
  df = df8_a[df8_a['Year']==Year].copy()
  if Cat == 'Recorded_results':
    fig8_a = px.box(data_frame=df, x=Period, y=Cat, color='Quarter',
                  title=f'Particle Size Results in {Year}', width=800, height=600)
    fig8_a.add_hrect(y0=24, y1=27) #, line_color="black" fillcolor="green", opacity=0.1)
  else:
    fig8_a = px.box(data_frame=df, x=Period, y=Cat, color='Quarter',
                  title=f'Fat Content % Results in {Year}', width=800, height=600)
    fig8_a.add_hline(y=54.5)
  fig8_a.show()

interactive(children=(Dropdown(description='Year', options=(2021, 2020, 2019, 2018, 2017), value=2021), Dropdo…

##Conch Interactive Plot

In [ ]:
#Conching Interactive visualization
@interact(Year=year_list, Period=period_list)
def plot_particle_size(Year, Period):
  df = df8_b[(df8_b['Year']==Year)].copy()
  fig8_b = px.box(data_frame=df, x=Period, y='Recorded_Results', color='Quarter',
                  title=f'Conch Malvern D90 in {Year}', width=400, height=600)
  fig8_b.add_hrect(y0=28, y1=30, line_color="black", fillcolor="red", opacity=0.2, annotation_text='High_zone')
  fig8_b.update_yaxes(range=[24,34])
  fig8_b.show()

interactive(children=(Dropdown(description='Year', options=(2021, 2020, 2019, 2018, 2017), value=2021), Dropdo…

##Quarterly Visualizations Interactive Plot

In [ ]:
#Creating quarterly visualization
@interact(Cat=cat_list, Quarter=quarter_list)
def plot_quarter_results(Cat, Quarter):
  df = df8[df8['Quarter']==Quarter].copy()
  if Cat == 'Recorded_results':
    fig8_c = px.box(data_frame=df, y=Cat, color='Year',
                    title=f'Particle Size Results in {Quarter} by Year', width=800, height=400)
  else:
    fig8_c = px.box(data_frame=df, y=Cat, color='Year',
                    title=f'Fat Content % Results in {Quarter} by Year', width=800, height=400)
  fig8_c.show()

interactive(children=(Dropdown(description='Cat', options=('Recorded_results', 'Fat_content_%'), value='Record…

##Histogram Interactive Plot

In [ ]:
#Creating Histogram visualization
@interact(Year=year_list)
def plot_marginal_histogram(Year):
  df8_c = df8[(df8['Year']==Year)].copy()
  fig8_c = px.histogram(data_frame=df8_c, x='Recorded_results', color='Quarter', marginal='box', histnorm='percent',
                        nbins=20, title=f'Histogram of Particle Size Results', width=800, height=450)
  fig8_c.show()

interactive(children=(Dropdown(description='Year', options=(2021, 2020, 2019, 2018, 2017), value=2021), Output…

##Empirical CDF Interactive Plot

In [ ]:
#Creating ECDF visualization
@interact(Year=year_list, Quarter=quarter_list)
def plot_ecdf_seaborn(Year, Quarter):
  df = df8[(df8['Year']==Year) & (df8['Quarter']==Quarter)].copy()
  sns.ecdfplot(data=df, x='Recorded_results', hue='Quarter', stat='proportion', palette=['purple'], complementary=False)
  plt.title(f'Empirical CDF of Particle Size in {Quarter} - {Year}')
  plt.show()

interactive(children=(Dropdown(description='Year', options=(2021, 2020, 2019, 2018, 2017), value=2021), Dropdo…

##BI-variate Interactive Plot

In [ ]:
#Plotting bi-variate interactive visualization
@interact(Year = year_list, Cat=['Malvern_avg', 'Fat_avg'])
def plot_particle_scatter(Year, Cat):
  sns.scatterplot(x='Weeknum', y='Conch_avg', data=df8_merged[df8_merged['Year']==Year])
  sns.scatterplot(x='Weeknum', y=Cat, data=df8_merged[df8_merged['Year']==Year])
  plt.title(f'Malvern Bi-Variate Analysis in {Year}')
  plt.show()

#2RR Analytics

## Uploading necessary packages

In [ ]:
#Upgrading Plotly
!pip install plotly --upgrade

In [ ]:
#Uploading packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from ipywidgets import interact

In [ ]:
#Customizing data visualization styles
sns.set_theme(rc = {'figure.figsize': (8, 5)})

In [ ]:
#Remove previous versions of the uploaded excel file
!rm 2021_2RR_particle_size_microns.xlsx

In [ ]:
#Uploading file from local drive ==> !rm file
from google.colab import files
uploaded9_a = files.upload()

Saving 2021_2RR_particle_size_microns.xlsx to 2021_2RR_particle_size_microns.xlsx


In [ ]:
#Storing dataset in a Pandas Dataframe
import io
df9_a = pd.read_excel(io.BytesIO(uploaded9_a['2021_2RR_particle_size_microns.xlsx']))

In [ ]:
#Checking the first dataframe info
df9_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   event      104 non-null    datetime64[ns]
 1   date       104 non-null    datetime64[ns]
 2   time       104 non-null    object        
 3   equipment  104 non-null    object        
 4   name       104 non-null    object        
 5   microns    104 non-null    int64         
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 5.0+ KB


##Data manipulation

In [ ]:
#Creating categories from datetime for first dataframe
df9_a = df9_a.assign(shift = np.where((df9_a['event'].dt.hour)<15, 'Day', 'Afternoon'),
                     day = df9_a['event'].dt.dayofweek, 
                     weeknum = df9_a['event'].dt.isocalendar().week)

In [ ]:
#Counting the values for different shifts
df9_a['shift'].value_counts()

Day          76
Afternoon    28
Name: shift, dtype: int64

In [ ]:
#Transforming numeric day into day of the week
days_dict = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
df9_a['day'].replace(days_dict, inplace=True)

##Interactive Visualization

In [ ]:
#Plotting the average microns vs time
@interact(Weeknum = df9_a['weeknum'].unique())
def plot_2RR_bar(Weeknum):
  df = df9_a[df9_a['weeknum']==Weeknum].copy()
  df = df.groupby(by=['day', 'shift']).agg(avg_microns = ('microns', 'mean')).reset_index()
  #df = df.sort_values(by='day')
  x_orders = {'day': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
              'shift': ['Day', 'Afternoon']}
  fig9_a = px.bar(data_frame=df, x='day', y='avg_microns', color='shift', barmode='group',
                  title=f'2RR average particle size in week {Weeknum}', height=500, width=800,
                  color_discrete_sequence=px.colors.qualitative.D3, category_orders=x_orders)
  fig9_a.add_hrect(y0=200, y1=250, line_color="black", fillcolor="red", opacity=0.2, annotation_text='High_zone')
  fig9_a.show()

interactive(children=(Dropdown(description='Weeknum', options=(46,), value=46), Output()), _dom_classes=('widg…

In [ ]:
#Plotting the variability of microns vs time
@interact(Weeknum = df9_a['weeknum'].unique())
def plot_2RR_microns(Weeknum):
  fig9_b = px.box(data_frame=df9_a, x='day', y='microns', color='shift',
                  title=f'2RR particle size microns in week {Weeknum}', height=500, width=800,
                  color_discrete_sequence=px.colors.qualitative.D3)
  fig9_b.add_hrect(y0=200, y1=250, line_color="black", fillcolor="red", opacity=0.2, annotation_text='High_zone')
  fig9_b.show()

interactive(children=(Dropdown(description='Weeknum', options=(46,), value=46), Output()), _dom_classes=('widg…

In [ ]:
#Creating the microns vs time
@interact(Weeknum = df9_a['weeknum'].unique())
def plot_2RR_histogram(Weeknum):
  fig9_c = px.histogram(data_frame=df9_a, x='microns', color='shift', marginal='box', histnorm='probability',
                        nbins=20, title=f'2RR particle microns histogram in week {Weeknum}', height=500, width=800,
                        color_discrete_sequence=px.colors.qualitative.D3)
  fig9_c.add_vline(x=200, line_color="red")
  fig9_c.show()

interactive(children=(Dropdown(description='Weeknum', options=(46,), value=46), Output()), _dom_classes=('widg…

#Fine Refiners Analytics

## Uploading necessary packages

In [ ]:
#Upgrading Plotly
!pip install plotly --upgrade

In [1]:
#Uploading packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from ipywidgets import interact

In [2]:
#Customizing data visualization styles
sns.set_theme(rc = {'figure.figsize': (8, 5)})

In [ ]:
#Remove previous versions of the uploaded excel file
!rm 2021_fine_refiners.xlsx

In [ ]:
#Uploading file from local drive ==> !rm file
from google.colab import files
uploaded3_a = files.upload()

In [5]:
#Storing dataset in a Pandas Dataframe
import io
df3_a = pd.read_excel(io.BytesIO(uploaded3_a['2021_fine_refiners.xlsx']))

In [6]:
#Checking the first dataframe info
df3_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            150 non-null    datetime64[ns]
 1   roll_num        150 non-null    object        
 2   fine_refiner_1  90 non-null     float64       
 3   fine_refiner_2  90 non-null     float64       
 4   fine_refiner_3  90 non-null     float64       
 5   fine_refiner_4  85 non-null     float64       
 6   fine_refiner_5  90 non-null     float64       
 7   fine_refiner_6  90 non-null     float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 9.5+ KB


##Data manipulation

In [8]:
#Creating categories from datetime for first dataframe
df3_a = df3_a.assign(day = df3_a['date'].dt.dayofweek,
                     weeknum = df3_a['date'].dt.isocalendar().week)

In [9]:
#Transforming numeric day into day of the week
days_dict = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
df3_a['day'].replace(days_dict, inplace=True)

In [10]:
#Melting the dataframe to long format
df3_a = pd.melt(df3_a, id_vars=['date', 'roll_num', 'weeknum'], value_vars=['fine_refiner_1', 'fine_refiner_2', 'fine_refiner_3', 'fine_refiner_4', 'fine_refiner_5', 'fine_refiner_6'],
        var_name='fine_refiner', value_name='temperature')

##Interactive Visualization

In [ ]:
#Plotting the average microns vs time
@interact(Weeknum = df3_a['weeknum'].unique())
def plot_fine_refiners(Weeknum):
  fig3_a = px.box(data_frame=df3_a, x='day', y='microns', 
                  title=f'Fine Refiners Tempearture in week {Weeknum}', height=500, width=800,
                  color_discrete_sequence=px.colors.qualitative.D3)
  #fig3_a.add_hrect(y0=200, y1=250, line_color="black", fillcolor="red", opacity=0.2, annotation_text='High_zone')
  fig3_a.show()